# hw6<br>




In [1]:
val sc = spark.sparkContext


In [2]:
import sys.process._

"tree datasets".!

val df = spark.read.option("header", "true").csv("datasets/ppkmSentiment/ppkm_dataset.csv")
df.show()


datasets
├── DoctorWho
│   ├── all-detailsepisodes.csv
│   ├── all-scripts.csv
│   ├── dwguide.csv
│   └── imdb_details.csv
├── HouseMDTranscripts
│   ├── season1.csv
│   ├── season2.csv
│   ├── season3.csv
│   ├── season4.csv
│   ├── season5.csv
│   ├── season6.csv
│   ├── season7.csv
│   └── season8.csv
├── IMDBSentimentAnalysis
│   ├── Test.csv
│   ├── Train.csv
│   └── Valid.csv
├── StarWars
│   ├── SW_EpisodeIV.txt
│   ├── SW_EpisodeV.txt
│   └── SW_EpisodeVI.txt
├── TopCompaniesTweets
│   ├── Company.csv
│   ├── Company_Tweet.csv
│   └── Tweet.csv
└── ppkmSentiment
    ├── ppkm_dataset.csv
    ├── ppkm_test.csv
    └── stopwordv1.txt

6 directories, 24 files
+-------+--------------------+
|  class|             comment|
+-------+--------------------+
|positif|Kami siap laksana...|
|positif|Siap melaksanakan...|
|positif|Siap dukung dan s...|
|positif|Langkah 3M ini su...|
|positif|Siap amankan selu...|
|positif|Siap utk di sosia...|
|positif|Mendukung kebijak...|
|positif|Mari ber

In [3]:
import org.apache.spark.ml.{Pipeline, PipelineModel}
import org.apache.spark.ml.classification.LogisticRegression
import org.apache.spark.ml.evaluation.MulticlassClassificationEvaluator
import org.apache.spark.ml.feature.{HashingTF, Tokenizer, IDF, StringIndexer}
import org.apache.spark.sql.types.{StringType, StructField, StructType}
import org.apache.spark.sql.{SparkSession, Dataset, Encoders}

case class TextLabel(label: String, text: String)
val sparkk = SparkSession.builder()
  .appName("IMDB Sentiment Analysis")
  .getOrCreate()

val schema = new StructType(Array(
  StructField("label", StringType, true),
  StructField("text", StringType, true)
))

import spark.implicits._
val trainData = sparkk.read.option("header", "true").schema(schema).csv("datasets/IMDBSentimentAnalysis/Train.csv").as[TextLabel]
val validData = sparkk.read.option("header", "true").schema(schema).csv("datasets/IMDBSentimentAnalysis/Valid.csv").as[TextLabel]


val tokenizer = new Tokenizer().setInputCol("text").setOutputCol("words")
val hashingTF = new HashingTF().setInputCol(tokenizer.getOutputCol).setOutputCol("rawFeatures").setNumFeatures(10000)
val idf = new IDF().setInputCol(hashingTF.getOutputCol).setOutputCol("features")
val labelStringIdx = new StringIndexer().setInputCol("label").setOutputCol("indexedLabel").setHandleInvalid("skip")t

val lr = new LogisticRegression().setLabelCol("indexedLabel").setFeaturesCol("features").setMaxIter(10).setRegParam(0.1) 

val pipeline = new Pipeline().setStages(Array(tokenizer, hashingTF, idf, labelStringIdx, lr))
val model = pipeline.fit(trainData)
val predictions = model.transform(validData)

val evaluator = new MulticlassClassificationEvaluator()
  .setLabelCol("indexedLabel")
  .setPredictionCol("prediction")
  .setMetricName("accuracy")

val accuracy = evaluator.evaluate(predictions)
println(s"Accuracy = $accuracy")

spark.stop()


org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 139.0 failed 1 times, most recent failure: Lost task 0.0 in stage 139.0 (TID 139, localhost, executor driver): org.apache.spark.SparkException: Failed to execute user defined function($anonfun$createTransformFunc$1: (string) => array<string>)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStage1.processNext(Unknown Source)
	at org.apache.spark.sql.execution.BufferedRowIterator.hasNext(BufferedRowIterator.java:43)
	at org.apache.spark.sql.execution.WholeStageCodegenExec$$anonfun$13$$anon$1.hasNext(WholeStageCodegenExec.scala:636)
	at scala.collection.Iterator$$anon$11.hasNext(Iterator.scala:409)
	at scala.collection.Iterator$$anon$11.hasNext(Iterator.scala:409)
	at scala.collection.Iterator$$anon$11.hasNext(Iterator.scala:409)
	at scala.collection.Iterator$class.foreach(Iterator.scala:891)
	at scala.collection.AbstractIterator.foreach(Iterator.scala:1334)
	at scala